# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 125894516, 'calculationPrice': 'tops', 'change': 0.13, 'changePercent': 0.00073, 'close': 0, 'closeSource': 'fiiaclfo', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 183.17, 'iexAskSize': 103, 'iexBidPrice': 182.7, 'iexBidSize': 415, 'iexClose': 182.02, 'iexCloseTime': 1672331132588, 'iexLastUpdated': 1678476389842, 'iexMarketPercent': 0.008649246637465845, 'iexOpen': 185.01, 'iexOpenTime': 1682573110833, 'iexRealtimePrice': 181, 'iexRealtimeSize': 101, 'iexVolume': 404560, 'lastTradeTime': 1655052044072, 'latestPrice': 182.47, 'latestSource': 'IEX real time price', 'latestTime': '3:02:23 PM', 'latestUpdate': 1649773913024, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 3031590159408, 'oddLotDelayedPr

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

16.69

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#  print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,162.330,50.27,N/A
1,AAL,18.845,-3.66,N/A
2,AAP,252.490,24.84,N/A
3,AAPL,181.920,16.34,N/A
4,ABBV,135.950,33.31,N/A
...,...,...,...,...
500,YUM,139.133,27.88,N/A
501,ZBH,128.640,33.75,N/A
502,ZBRA,623.698,39.86,N/A
503,ZION,66.300,6,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,16.630,3.22,N/A
1,BIO,769.070,3.6,N/A
2,HPQ,39.850,3.61,N/A
3,COO,423.180,3.64,N/A
4,EBAY,69.600,3.77,N/A
5,NRG,44.680,4.41,N/A
6,MPC,65.350,4.65,N/A
7,BEN,34.100,4.78,N/A
8,DHI,111.930,4.84,N/A
9,TSN,90.280,5.22,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Share to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy,Number of Share to Buy
0,HPE,16.630,3.22,N/A,12026.0
1,BIO,769.070,3.6,N/A,260.0
2,HPQ,39.850,3.61,N/A,5018.0
3,COO,423.180,3.64,N/A,472.0
4,EBAY,69.600,3.77,N/A,2873.0
5,NRG,44.680,4.41,N/A,4476.0
6,MPC,65.350,4.65,N/A,3060.0
7,BEN,34.100,4.78,N/A,5865.0
8,DHI,111.930,4.84,N/A,1786.0
9,TSN,90.280,5.22,N/A,2215.0


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()


#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
    
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url =  f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NAN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NAN       
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            
        ],
        index = rv_columns),
            ignore_index = True
        )
        
 

In [13]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,168.080,N/A,51.27,N/A,9.97,N/A,8.17,N/A,31.852042,N/A,14.979077,N/A,N/A
1,AAL,18.078,N/A,-3.7,N/A,-1.69,N/A,0.4987,N/A,-7.415877,N/A,1.458831,N/A,N/A
2,AAP,243.632,N/A,25.23,N/A,4.8,N/A,1.38,N/A,13.026294,N/A,3.042065,N/A,N/A
3,AAPL,186.120,N/A,16.15,N/A,23.82,N/A,4.08,N/A,12.464039,N/A,9.864359,N/A,N/A
4,ABBV,138.310,N/A,33.06,N/A,18.2,N/A,4.4,N/A,11.407993,N/A,7.848024,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,142.143,N/A,27.08,N/A,-5.15,N/A,6.42,N/A,21.990164,N/A,10.667396,N/A,N/A
501,ZBH,130.560,N/A,33.52,N/A,2.2,N/A,3.5,N/A,17.813650,N/A,5.802905,N/A,N/A
502,ZBRA,623.158,N/A,39.43,N/A,11.48,N/A,6.07,N/A,29.405934,N/A,12.645944,N/A,N/A
503,ZION,64.640,N/A,6,N/A,1.37,N/A,2.9,N/A,4.885815,N/A,2.547864,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]
#len(rv_dataframe[rv_dataframe.isnull().any(axis=1)]) to find number value of how many rows are missing data

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,187.700,N/A,60.77,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,307.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,312.020,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.21,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,23.530,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.920,N/A,14.86,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.210,N/A,32.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.370,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2996.850,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,104.480,N/A,35.41,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [19]:
for column in ['Price','Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [15]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [20]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [53]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
    
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row,metric])/100
rv_dataframe



,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,25.200,7936,6.63,0.42,0.4607,0.16,0.3978,0.18,3.074667,0.12,0.373370,0.04,4.554455
1,AAL,18.078,11063,-3.70,0.16,-1.6900,0.14,0.4987,0.24,-7.415877,0.06,1.458831,0.42,4.910891
2,HPQ,39.960,5005,3.73,0.2,-12.9600,0.06,0.3321,0.06,4.177158,0.32,1.769444,0.52,5.227723
3,PRU,111.550,1792,6.04,0.38,0.6758,0.24,0.6041,0.26,4.540413,0.38,0.611457,0.06,5.742574
4,BEN,35.300,5665,4.78,0.24,0.7770,0.36,1.0200,0.55,3.283195,0.16,0.862133,0.14,6.772277
5,KSS,50.950,3925,8.11,0.58,1.4000,0.72,0.3591,0.14,3.789334,0.22,1.199913,0.36,7.960396
6,BA,210.660,949,-14.31,0.1,-8.6300,0.12,1.9800,0.92,-32.808859,0.02,-1366.768339,0.02,8.277228
7,HPE,16.620,12033,3.26,0.18,0.5251,0.2,0.3837,0.16,7.556057,0.86,2.040756,0.66,8.475248
8,AIG,58.055,3445,9.14,0.6,0.7415,0.32,1.0356,0.58,4.462117,0.34,0.970426,0.2,8.514851
9,TSN,89.750,2228,5.41,0.28,0.8983,0.42,0.3453,0.11,3.897302,0.28,3.394077,0.88,8.693069


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [54]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,25.200,7936,6.63,0.42,0.4607,0.16,0.3978,0.18,3.074667,0.12,0.373370,0.04,0.184
1,AAL,18.078,11063,-3.70,0.16,-1.6900,0.14,0.4987,0.24,-7.415877,0.06,1.458831,0.42,0.204
2,HPQ,39.960,5005,3.73,0.2,-12.9600,0.06,0.3321,0.06,4.177158,0.32,1.769444,0.52,0.232
3,PRU,111.550,1792,6.04,0.38,0.6758,0.24,0.6041,0.26,4.540413,0.38,0.611457,0.06,0.264
4,BEN,35.300,5665,4.78,0.24,0.7770,0.36,1.0200,0.55,3.283195,0.16,0.862133,0.14,0.29
5,KSS,50.950,3925,8.11,0.58,1.4000,0.72,0.3591,0.14,3.789334,0.22,1.199913,0.36,0.404
6,BA,210.660,949,-14.31,0.1,-8.6300,0.12,1.9800,0.92,-32.808859,0.02,-1366.768339,0.02,0.236
7,HPE,16.620,12033,3.26,0.18,0.5251,0.2,0.3837,0.16,7.556057,0.86,2.040756,0.66,0.412
8,AIG,58.055,3445,9.14,0.6,0.7415,0.32,1.0356,0.58,4.462117,0.34,0.970426,0.2,0.408
9,TSN,89.750,2228,5.41,0.28,0.8983,0.42,0.3453,0.11,3.897302,0.28,3.394077,0.88,0.394


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [55]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace =True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,25.200,7936,6.63,0.42,0.4607,0.16,0.3978,0.18,3.074667,0.12,0.373370,0.04,0.184
1,AAL,18.078,11063,-3.70,0.16,-1.6900,0.14,0.4987,0.24,-7.415877,0.06,1.458831,0.42,0.204
2,HPQ,39.960,5005,3.73,0.2,-12.9600,0.06,0.3321,0.06,4.177158,0.32,1.769444,0.52,0.232
3,BA,210.660,949,-14.31,0.1,-8.6300,0.12,1.9800,0.92,-32.808859,0.02,-1366.768339,0.02,0.236
4,PRU,111.550,1792,6.04,0.38,0.6758,0.24,0.6041,0.26,4.540413,0.38,0.611457,0.06,0.264
5,BEN,35.300,5665,4.78,0.24,0.7770,0.36,1.0200,0.55,3.283195,0.16,0.862133,0.14,0.29
6,TSN,89.750,2228,5.41,0.28,0.8983,0.42,0.3453,0.11,3.897302,0.28,3.394077,0.88,0.394
7,UAL,46.110,4337,-4.40,0.14,2.8000,0.98,0.7690,0.38,-7.837867,0.04,1.484338,0.44,0.396
8,AIZ,154.790,1292,7.22,0.46,1.5900,0.84,0.8912,0.44,1.874326,0.08,0.867094,0.16,0.396
9,KSS,50.950,3925,8.11,0.58,1.4000,0.72,0.3591,0.14,3.789334,0.22,1.199913,0.36,0.404


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [56]:
portfolio_input()

Enter the value of your portfolio:10000000


In [57]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,25.200,7936,6.63,0.42,0.4607,0.16,0.3978,0.18,3.074667,0.12,0.373370,0.04,0.184
1,AAL,18.078,11063,-3.70,0.16,-1.6900,0.14,0.4987,0.24,-7.415877,0.06,1.458831,0.42,0.204
2,HPQ,39.960,5005,3.73,0.2,-12.9600,0.06,0.3321,0.06,4.177158,0.32,1.769444,0.52,0.232
3,BA,210.660,949,-14.31,0.1,-8.6300,0.12,1.9800,0.92,-32.808859,0.02,-1366.768339,0.02,0.236
4,PRU,111.550,1792,6.04,0.38,0.6758,0.24,0.6041,0.26,4.540413,0.38,0.611457,0.06,0.264
5,BEN,35.300,5665,4.78,0.24,0.7770,0.36,1.0200,0.55,3.283195,0.16,0.862133,0.14,0.29
6,TSN,89.750,2228,5.41,0.28,0.8983,0.42,0.3453,0.11,3.897302,0.28,3.394077,0.88,0.394
7,UAL,46.110,4337,-4.40,0.14,2.8000,0.98,0.7690,0.38,-7.837867,0.04,1.484338,0.44,0.396
8,AIZ,154.790,1292,7.22,0.46,1.5900,0.84,0.8912,0.44,1.874326,0.08,0.867094,0.16,0.396
9,KSS,50.950,3925,8.11,0.58,1.4000,0.72,0.3591,0.14,3.789334,0.22,1.199913,0.36,0.404


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [58]:
writer = pd.ExcelWriter('Value Strategy.xlsx', engine ='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [59]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [65]:
column_formats= {
                    'A':['Ticker', string_template],
                    'B':['Price', dollar_template],
                    'C':['Number of Shares to Buy', integer_template],
                    'D':['Price-to-Earnings Ratio', float_template],
                    'E':['PE Percentile', percent_template],
                    'F':['Price-to-Book Ratio', float_template],
                    'G':['PB Percentile', percent_template],
                    'H':['Price-to-Sales Ratio', float_template],
                    'I':['PS Percentile', percent_template],
                    'J':['EV/EBITDA', float_template],
                    'K':['EV/EBITDA Percentile', percent_template],
                    'L':['EV/GP', float_template],
                    'M':['EV/GP Percentile', percent_template],
                    'N':['RV Score', percent_template],
                }
for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [66]:
writer.save()